WORK_IN_PROGRESS

<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://assets.vespa.ai/logos/Vespa-logo-green-RGB.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg">
  <img alt="#Vespa" width="200" src="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg" style="margin-bottom: 25px;">
</picture>

# pyvespa Query builder

The Query Builder in pyvespa provides a more 'pythonic' way of building YQL query strings and this guide goes through constructing queries for the [Covid-19 Vespa instance](https://cord19.vespa.ai/) and the [vespa search instance](https://api.search.vespa.ai)

While simple queries can be built with string formatting; building complex queries that contain a lot of dynamic parameters can get difficult and this is where the query builder comes in handy.

The query builder is only responsible to build a YQL string which then can be used for querying using various methods (see query notebook).

First lets install pyvespa that supports query builder:

In [ ]:
!pip install pyvespa>=0.52.0

In [2]:
from vespa.application import Vespa
from vespa import querybuilder as qb

app = Vespa(url="https://api.cord19.vespa.ai")

At an abstract level a query can be divided into 2 parts: 
 - `QueryField`: the fields part of the schema and needs to be queried on. For example in the Covid-19 schema `title`, `abstract`, etc can be used as fields.
 - `Condition`: Logic operators used to combine the queryfields (and, or, etc)

First lets take a look at the schema of the documents we will be querying on. And the best way to do this, is to use the `True` condition:

In [ ]:
yql_query = qb.select("*").from_("doc").where(True)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits[0]["fields"].keys()

Do note that retrival from multiple sources can also be done if a tuple is provied to `from_()`: example: `from_(sd1, sd2) `.

Lets start with something simple: building a query for fetching documents with a specific keyword in the title.


In [ ]:
title_field = qb.QueryField("title")
condition = title_field.contains("vaccine")
yql_query = qb.select(["title", "abstract"]).from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

While the `select()` accepts a list of QueryFields you can also give a string.

In [ ]:
yql_query = qb.select("title, abstract").from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

The same query can be extented by chaining it with the `set_timeout()` to change the default timeout:

In [ ]:
yql_query = qb.select("title, abstract").from_("doc").where(condition).set_timeout(70)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

Which can then further be chained with other parameters like `limit()`, `orderByAsc()` or just `add_parameter()`; And you don't have to worry about the order of the function chain because the query builder will safely handel the order and generate a valid YQL string. Some parameter options:
 - `set_offset()`
 - `set_limit()`
 - `set_timeout()`
 - `orderByDesc()`
 - `orderByAsc()`
 - `add_parameter()`

In [ ]:
yql_query = (
    qb.select("title, abstract")
    .from_("doc")
    .where(condition)
    .set_timeout(70)
    .set_limit(2)
    .orderByAsc(qb.QueryField("timestamp"))
)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=2)
resp.hits

And when there is a need to add some annotations to a conditions or a field; you can do that with `annotate()`


In [ ]:
condition = title_field.contains("vaccine")
annotated_condtition = condition.annotate({"highlight": True})
yql_query = qb.select("title").from_("doc").where(annotated_condtition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

You can also invert conditions after building them by just adding the `~` symbol in front of the condition.

In [ ]:
condition = ~condition
yql_query = qb.select("title, abstract").from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

Now lets say you wanted to query over multiple fields and combine them in a logical way along with mixing, matching logic operators:
 - `all()` for logical AND.
 - `any()` for local OR.

In [ ]:
title = qb.QueryField("title")
abstract = qb.QueryField("abstract")
source = qb.QueryField("source")

condition_1 = title.contains("vaccine")
condition_2 = abstract.contains("bad")
condition_3 = source.contains("PMC")
condition_4 = title.contains("anti")
condition = qb.all(condition_1, ~condition_2, qb.any(condition_3, ~condition_4))

yql_query = qb.select(["title", "abstract"]).from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

You can also build the same query using the logical operators without `any()` or `all()` but by using:
 -  `&` for AND
 -  `|` for OR

In [ ]:
title = qb.QueryField("title")
abstract = qb.QueryField("abstract")
source = qb.QueryField("source")

condition_1 = title.contains("vaccine")
condition_2 = abstract.contains("bad")
condition_3 = source.contains("PMC")
condition_4 = title.contains("anti")

condition = (condition_1 & ~condition_2) & (condition_3 | ~condition_4)
yql_query = qb.select(["title", "abstract"]).from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

When you do want to query over numeric fields. Query builder provides both: the logical operator method (`>`, `<` and `=`) or the comparison methods: `ge()`, `le()` or `eq()`

Or just use `in_range()` which can be used to achieve similar results

In [ ]:
timestamp = qb.QueryField("timestamp")
id_filed = qb.QueryField("id")

condition_1 = (timestamp > 1554501500) & (timestamp < 1554501700)
condition_2 = id_filed.in_range(11105, 11305)
condition_3 = timestamp.eq(0)
condition = qb.any(condition_1, condition_2) | condition_3
yql_query = qb.select(["title", "abstract"]).from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=2)
resp.hits

And if you dont like either of these approches, you can just use regex/fuzzy matching using `matches()`:

In [ ]:
condition = qb.QueryField("title").matches("covid")
yql_query = qb.select(["title", "abstract"]).from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, hits=1)
resp.hits

The true power comes when you build each condition separately, enhancing long-term maintainability and readability.

QB also offers a `userQuery()` which can be used as a condition for when you want to keep the query itself independent.

In [ ]:
condition = qb.userQuery()
yql_query = qb.select("title, abstract").from_("doc").where(condition)
print(f"Built query: {yql_query}")
resp = app.query(yql=yql_query, query="guards adults too, for now", hits=1)
resp.hits

Query builder also provides a `in_()` which can be used to query multiple values in a single field.

In [ ]:
id_field = qb.QueryField("id")
condition = id_field.in_(11205, 81788, 117939, 287995)
yql_query = qb.select("title, id").from_("doc").where(condition)
resp = app.query(yql=yql_query, hits=4)
resp.hits

And the same method can be used if the query field was of a string data type:

In [ ]:
source_field = qb.QueryField("source")
condition = source_field.in_("PMC", "Medline")
yql_query = qb.select("title, source").from_("doc").where(condition)
resp = app.query(yql=yql_query, hits=2)
resp.hits

The query builder provides the `wand()` interface along with adding appropriate annotations as well (refer [wand documentation](hhttps://docs.vespa.ai/en/using-wand-with-vespa.html#wand)):


In [ ]:
condition = qb.wand(
    "title",
    weights={"alphavirus": 1, "vaccine": 2},
    annotations={"scoreThreshold": 0.13, "targetHits": 2},
)
yql_query = qb.select("title, source").from_("doc").where(condition)
resp = app.query(yql=yql_query, hits=2)

resp.hits


Now thats how you do the basic querying using the query builder. But obviously, the real world queries tend to be very complex with grouping, nearest Neighbour, dot products etc. And the query builder provides a clean interface for all of them as well. Again, it only generates a syntactically correct YQL string. All checks like using the right field, or the operator will have to be managed on the cleint side.

For the usage of these methods, we move to the Vespa Search application which has a much more complex schema:

In [ ]:
search_app = Vespa(url="https://api.search.vespa.ai")


resp = search_app.query(yql="select * from documentation where true limit 2")
resp.hits


In [ ]:
condition = qb.nearestNeighbor(
    field="text_embedding", query_vector="q", annotations={"targetHits": 5}
)

yql_query = qb.select("*").from_("documentation").where(condition)
print(f"Built query: {yql_query}")

resp = search_app.query(
    yql=yql_query,
    ranking="hybrid2",
    query="How can the query builder be used to create a query?",
    body={"input.query(q)": "embed(@query)"},
)
resp.hits


Now all of these operations can also be combined with the `nearestNeighbor()`. The `annotations` can be used to modify the behaviour. And this one is always required because this is also how we set the `targetHits` which defaults to 10. The query builder is a perfect fit here because to achieve the same YQL query with python formatting you will have to escape the curly brackets which sometimes can become messy. Now all of that is replaced with a clean API.

In [ ]:
# Find messages where terms appear close to each other
text = qb.QueryField("text")
condition = text.contains(qb.near("send", "qps", distance=5))
yql_query = qb.select(["message_id", "text"]).from_("documentation").where(condition)
print(f"Built query: {yql_query}")

resp = search_app.query(yql=yql_query, hits=2)
resp.hits
